# Demonstration RAG-Eval


Quick test: see if things work as planned

In [1]:
from rag_eval.protocol import LLMClient
from rag_eval.client import AzureClient
from rag_eval.components import ClaimExtractor, EntailmentJudge
from rag_eval.data_models import EvalContainer, ClaimVerdict
from rag_eval.evaluators import AnswerPrecisionEvaluator, RetrievalPrecisionEvaluator
from rag_eval.coordinator import RAGEvaluator
from rag_eval.helpers import _safe_json
import json
from data.rag_eval_samples import test_examples
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

c:\Users\ANDERBE\AI-Agent-POC-git\AI-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


# Test Agent

Check if agent module actually works.

In [4]:
# load agent and pipeline for indexing docs
import agent.main as main
from rag.indexing import pdf_files, index_pipeline

In [5]:
index_pipeline.run({"converter": {"sources": pdf_files}})

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
Batches: 100%|██████████| 2/2 [00:00<00:00,  2.47it/s]


{'writer': {'documents_written': 51}}

In [5]:
main.food_agent(user_input="hi")


▶️ Step 1: thinking…
🍕 Fooder: Hello! I'm Fooder, your food ordering agent. 🍔🍕🥤

Would you like to see the available food options or place an order today? Let me know how I can help you!


## RAG-Loop
Loop which passes questions to agent and extracts generated answers alongside retrieved contexts (plain text). Combines them with ground truth and query to form `EvalContainer`-object.

In [6]:
# quick test of my query_gt_extractor function
from src.rag_eval.helpers import query_gt_extractor

queries, gt = query_gt_extractor(path="data/agent_eval_questions.json", tokenize=False)
print(gt)


['2010', 'La cucina povera', 'Auguste\u202fEscoffier', 'Jerry\u202fThomas', '1920\u202f–\u202f1933', 'Orecchiette', 'Tarte\u202fTatin', '8,50\u202f€', 'Horchata', 'Secondo', 'Béchamel', 'Reis', 'Mezcal', 'Churros', '2,00\u202f€', 'Risotto\u202fal\u202fTartufo\u202fNero\u202fdel\u202fMastino', 'Mole', 'Camembert', 'Tiramisù\u202fClassico\u202f&\u202fCoda\u202fdi\u202fVolpe', 'Rindfleisch', 'Madrid', 'R', '1969', '7', 'Ernest\xa0Hemingway', 'Mais, Bohnen, Chili, Kürbis, Kakao; UNESCO‑Jahr\xa02010', 'Mexikanische Küche\xa02010; Gastronomisches Mahl der Franzosen\xa02010', 'Parmigiano\u202fReggiano, Mozzarella\u202fdi\u202fBufala; Camembert, Roquefort', 'Reis (Risotto) für Norditalien; Olivenöl für die Provence', 'Prosecco Extra Dry del Cucciolo\u202f–\u202f50,00\u202f€; Vin Rouge Français\u202f(0,75\u202fl)\u202f–\u202f18,00\u202f€', 'Martini, Manhattan, Daiquiri; Negroni', 'Elote – gegrillter Mais mit Limette und Käse; Caprese (Mozzarella, Tomate, Basilikum)', 'Béchamel; Filetto\u202fdi\

This code cell creates a RAG-loop for the agent. It:  
- feeds the questions from the test battery to the agent  
- extracts the generated answer and retrieved context from the RAG-call
- returns a list of `EvalContainer`-objects storing question, ground truth, generated answer and retrieved context

It thus lays the foundation for the `RAGEevaluator`. 

In [25]:
from src.rag_eval.helpers import query_gt_extractor
from rag_eval.data_models import EvalContainer
# force reload of agent because I update it often
import importlib
import agent.main as main
importlib.reload(main)  # forces reload right now


def rag_loop_agent() -> list[EvalContainer]:
    eval_conts = []
    queries, gts = query_gt_extractor(path="data/agent_eval_questions.json", tokenize=False) # get questions and ground truth
    for idx, (q, gt) in enumerate(zip(queries, gts)): # maybe we can do without zip and do not need to iterate over gts because I only need idx to access the right gt
        result = main.food_agent(user_input=q)

        if result: 
            # extract what is needed for EvalContainer-object
            query = queries[idx]
            ground_truth_answer = gts[idx]
            retrieved_texts = result[1]
            generated_answer = result[0]

            eval_cont = EvalContainer(query=query,
                                      ground_truth_answer=ground_truth_answer,
                                      generated_answer=generated_answer[0], # unlist generated_answer
                                      retrieved_texts=retrieved_texts[0] # unnest list with retrieved texts
                                      )
            
            eval_conts.append(eval_cont)

    return eval_conts
        

    





In [26]:

result = rag_loop_agent()


▶️ Step 1: thinking…

▶️ Step 1: thinking…

▶️ Step 1: thinking…

▶️ Step 1: thinking…
TOOL CALL ▶ execute_agentic_rag {"question":"Wer gilt laut Cocktail‑Dokument als Vater der modernen Barkultur?"}
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.84it/s]


(['Laut Cocktail-Dokument gilt Jerry Thomas als Vater der modernen Barkultur.'], [['erlebte die Barkultur eine Renaissance. Mixologen begannen, sich wieder\nauf Qualitaet, Balance und handwerkliche Praezision zu konzentrieren. Die "Craft\nCocktail"-Bewegung setzte sich gegen Fertigmischungen und Sirup-Cocktails durch und brachte\nKlassiker zurueck auf die Karte. Barkeeper wurden zu Kuenstlern - mit Kraeutern, Rauch, Infusionen\nund selbstgemachten Zutaten kreierten sie neue Geschmackserlebnisse. Weltweit entstanden Bars\nmit thematischen ', 'Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktails auch als kulturelle Artefakte verstanden: Der Negroni\nstammt aus Italien, der Caipirinha aus Brasilien, der Mojito aus Kuba. Trends wie alkoholfreie\nCocktai

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.79it/s]


(['Die Prohibition in den USA dauerte laut Cocktail-Dokument von 1920 bis 1933.'], [['Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Geschichte des Cocktail-Mixings\nUeber das Land\nDie Geschichte des Cocktail-Mixings ist eng mit gesellschaftlichen, wirtschaftlichen und politischen\nEntwicklungen verbunden. Erste schriftliche Erwaehnungen des Begriffs "Cocktail" tauchten Anfang\ndes 19. Jahrhunderts in den USA auf - definiert als "stimulierendes Getraenk aus Spirituosen,\nZucker, Wasser und Bitters". Das sogenannte "Golden Age of Cocktails" begann in der Mitte ', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktails auch als kulturelle Artefakte verstanden: Der Negroni\nstammt aus Italien, der Caipirinha aus Brasilien, der Mojito aus Kuba. Trends wie alkoholfreie\nCocktails ("Mocktails"), lokale

...

(['**Carne Asada Taco:**  \nDie drei Bestandteile des Carne Asada Taco aus dem Tönis‑Tacos‑Menü sind:  \n- Rindfleisch  \n- Zwiebeln  \n- Koriander  \n\n**Pizza «Doggish Special»:**  \nDer vollständige Name und die Zutaten der Pizza «Doggish Special» sind:  \nDoggish Special (Tomate, Mozzarella, Rinderhack, Speck, Zwiebeln)'], [['Tönis Tacos\nVorspeisen\nGuacamole mit Tortilla-Chips - 5,50 €\nQuesadillas mit Käse und Jalapeños - 6,00 €\nMexikanische Bohnen-Suppe - 4,50 €\nElote – gegrillter Mais mit Limette und Käse - 5,00 €\nTacos\nCarne Asada Taco (Rindfleisch, Zwiebeln, Koriander) - 3,50 €\nPollo Taco (gegrilltes Hähnchen, Pico de Gallo) - 3,50 €\nVeggie Taco (gegrilltes Gemüse, Avocado) - ', "Anton's Pizza - Doggish Delights\nVorspeisen\nBruschetta mit Tomaten und Basilikum - 5,50 €\nKnoblauchbrot mit Kräuterbutter - 4,00 €\nCaprese (Mozzarella, Tomate, Basilikum) - 6,50 €\nFrittierte Mozzarella-Sticks - 6,00 €\nPizza\nMargherita (Tomate, Mozzarella, Basilikum) - 8,50 €\nDoggish Sp

Batches: 100%|██████████| 1/1 [00:00<00:00, 12.65it/s]


(['Im Tönis-Tacos-Menü gibt es folgende Bohnenbeilagen:\n\n- Mexikanische Bohnen-Suppe\n- Refried Beans (zerdrückte Bohnen)\n- Bohnen als Bestandteil von Burritos (Classic Burrito mit Reis, Bohnen und Rindfleisch; Veggie Burrito mit Guacamole enthält vermutlich auch Bohnen)\n\nEine davon nutzt eine der „heiligen Fünf“ Zutaten der mexikanischen Küche, nämlich Bohnen. Die „heiligen Fünf“ sind: Mais, Bohnen, Chili, Kürbis und Tomaten. Bohnen sind also eine der zentralen Grundzutaten und finden sich bei Tönis Tacos in den Beilagen und Burritos wieder.'], [['Tönis Tacos\nVorspeisen\nGuacamole mit Tortilla-Chips - 5,50 €\nQuesadillas mit Käse und Jalapeños - 6,00 €\nMexikanische Bohnen-Suppe - 4,50 €\nElote – gegrillter Mais mit Limette und Käse - 5,00 €\nTacos\nCarne Asada Taco (Rindfleisch, Zwiebeln, Koriander) - 3,50 €\nPollo Taco (gegrilltes Hähnchen, Pico de Gallo) - 3,50 €\nVeggie Taco (gegrilltes Gemüse, Avocado) - ', '€\nRefried Beans (zerdrückte Bohnen) - 3,00 €\nSalsa Trio (Mild, M

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.36it/s]


(['Das Meeresfrüchte-Risotto im Anton’s-Pizza-Menü heißt:  \n**Meeresfrüchte-Risotto**.'], [["Mozzarella, Paprika, Zucchini, Oliven) - 10,00 €\nPasta\nSpaghetti Bolognese - 9,50 €\nTagliatelle mit Pesto und Pinienkernen - 10,00 €\nPenne Arrabiata (pikant) - 8,50 €\nLasagne al Forno - 11,00 €\x0cAnton's Pizza - Doggish Delights\nRisotto\nRisotto ai Funghi (mit Pilzen) - 11,50 €\nMeeresfrüchte-Risotto - 14,00 €\nRisotto Primavera (Gemüse der Saison) - 12,00 €\nGetränke\nMineralwasser (0,5 ", "Anton's Pizza - Doggish Delights\nVorspeisen\nBruschetta mit Tomaten und Basilikum - 5,50 €\nKnoblauchbrot mit Kräuterbutter - 4,00 €\nCaprese (Mozzarella, Tomate, Basilikum) - 6,50 €\nFrittierte Mozzarella-Sticks - 6,00 €\nPizza\nMargherita (Tomate, Mozzarella, Basilikum) - 8,50 €\nDoggish Special (Tomate, Mozzarella, Rinderhack, Speck, Zwiebeln) - 11,50 €\nFunghi (Tomate, Mozzarella, Champignons) - 9,50 €\nVeggie Delight (Tomate, ", 'Chez Anton - Délices Canins\nEntrées\nCroquettes d’Appétit (Knus

In [27]:
print(result)

[EvalContainer(query='Wer gilt laut Cocktail‑Dokument als Vater der modernen Barkultur?', ground_truth_answer='Jerry\u202fThomas', generated_answer='Laut Cocktail-Dokument gilt Jerry Thomas als Vater der modernen Barkultur.', retrieved_texts=['erlebte die Barkultur eine Renaissance. Mixologen begannen, sich wieder\nauf Qualitaet, Balance und handwerkliche Praezision zu konzentrieren. Die "Craft\nCocktail"-Bewegung setzte sich gegen Fertigmischungen und Sirup-Cocktails durch und brachte\nKlassiker zurueck auf die Karte. Barkeeper wurden zu Kuenstlern - mit Kraeutern, Rauch, Infusionen\nund selbstgemachten Zutaten kreierten sie neue Geschmackserlebnisse. Weltweit entstanden Bars\nmit thematischen ', 'Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktail

## Eval Loop
Extract individual queries from `EvalContainer-object` like so: 
```
container = test_examples[1]
print(container.query)
print(container)
```

In [29]:
res = result[0]
print(res.retrieved_texts)

['erlebte die Barkultur eine Renaissance. Mixologen begannen, sich wieder\nauf Qualitaet, Balance und handwerkliche Praezision zu konzentrieren. Die "Craft\nCocktail"-Bewegung setzte sich gegen Fertigmischungen und Sirup-Cocktails durch und brachte\nKlassiker zurueck auf die Karte. Barkeeper wurden zu Kuenstlern - mit Kraeutern, Rauch, Infusionen\nund selbstgemachten Zutaten kreierten sie neue Geschmackserlebnisse. Weltweit entstanden Bars\nmit thematischen ', 'Class" oder die "Tales of the\nCocktail" zeichnen jaehrlich neue Entwicklungen aus. Damit ist das Mixen von Cocktails heute\nsowohl Handwerk als auch Kunstform - mit globaler Anerkennung.', 'Schwerpunkten, etwa Tiki-Bars, Speakeasies oder molekulare\nMixologie-Konzepte. Heute werden Cocktails auch als kulturelle Artefakte verstanden: Der Negroni\nstammt aus Italien, der Caipirinha aus Brasilien, der Mojito aus Kuba. Trends wie alkoholfreie\nCocktails ("Mocktails"), lokale Zutaten oder die Rueckkehr zu simplen Rezepten (z. B. Gin

In [30]:
# initiate llm 
api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


# run evaluator
coord = RAGEvaluator(llm_extractor=llm, llm_judge=llm)
data = result[0:2]


for i, res in enumerate(data, start=1):
    print(f"EXAMPLE {i}")
    results = coord.evaluate_all(res)
    print(json.dumps(results, indent=2))


EXAMPLE 1
{
  "answer": {
    "gt_claims": [
      "Jerry Thomas is a person."
    ],
    "gen_claims": [
      "Jerry Thomas gilt als Vater der modernen Barkultur.",
      "Diese Information stammt aus dem Cocktail-Dokument."
    ],
    "gt_claim_recall": 1.0,
    "hallucination_precision": 0.0,
    "gt_claim_verdicts": [
      {
        "claim": "Jerry Thomas is a person.",
        "verdict": "supported",
        "rationale": "the evidence refers to jerry thomas as the 'vater der modernen barkultur' (father of modern bar culture), which implies that jerry thomas is a person. therefore, the claim is supported.",
        "doc_indices": null
      }
    ],
    "gen_claim_verdicts": [
      {
        "claim": "Jerry Thomas gilt als Vater der modernen Barkultur.",
        "verdict": "not_enough_info",
        "rationale": "the evidence only provides the name 'jerry thomas' without any information about his contributions or status in the context of modern bar culture. therefore, it is not 

## Ordinary Agent Test
Just to see if documents have been indexed and are accessible

In [7]:
import agent.backup.backup_main as backup
backup.food_agent()


▶️ Step 1: thinking…
TOOL CALL ▶ execute_plain_rag {"question":"Where can I get tacos and which types are available?"}
Invoked plain RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.71it/s]



▶️ Step 2: thinking…
🍕 Fooder: You can get tacos from Tönis Tacos. The available taco options are:

- Carne Asada Taco (Rindfleisch, Zwiebeln, Koriander) - 3,50 €
- Pollo Taco (gegrilltes Hähnchen, Pico de Gallo) - 3,50 €
- Veggie Taco (gegrilltes Gemüse, Avocado)

Would you like to order any of these tacos, or do you want more details about the menu?
🍕 Fooder: Goodbye 🍕
